# Word Sense Disambiguation using Neural Networks
Adam Ek

----------------------------------

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

----------------------------------

A problem with static distributional vectors is the difficulty of distinguishing between different *word senses*. We will continue our exploration of word vectors by considering *trainable vectors* or *word embeddings* for Word Sense Disambiguation (WSD).

The goal of word sense disambiguation is to train a model to find the sense of a word (homonyms of a word-form). For example, the word "bank" can mean "sloping land" or "financial institution". 

(a) "I deposited my money in the **bank**" (financial institution)

(b) "I swam from the river **bank**" (sloping land)

In case a) and b) we can determine that the meaning of "bank" based on the *context*. To utilize context in a semantic model we use *contextualized word representations*. Previously we worked with *static word representations*, i.e. the representation does not depend on the context. To illustrate we can consider sentences (a) and (b), the word **bank** would have the same static representation in both sentences, which means that it becomes difficult for us to predict its sense. What we want is to create representations that depend on the context, i.e. *contextualized embeddings*. 

We will create contextualized embeddings with Recurrent Neural Networks. You can read more about recurrent neural netoworks [here](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). Your overall task in this lab is to create a neural network model that can disambiguate the word sense of 30 different words. 

In [2]:
# first we import some packages that we need
import torch
import torch.nn as nn
import torchtext
import numpy as np
import pandas as pd
import torch.optim as optim
from torchtext.legacy.data import Field, BucketIterator, Iterator, TabularDataset, Dataset
import numpy as np

# our hyperparameters (add more when/if you need them)
#device = torch.device('cuda:0')
device = torch.device('cpu')

batch_size = 8
learning_rate = 0.001
epochs = 3

# 1. Working with data

A central part of any machine learning system is the data we're working with. In this section we will split the data (the dataset is located here: ``wsd-data/wsd_data.txt``) into a training set and a test set. We will also create a baseline to compare our model against. Finally, we will use TorchText to transform our data (raw text) into a convenient format that our neural network can work with.

## Data

The dataset we will use contain different word sense for 30 different words. The data is organized as follows (values separated by tabs): 
- Column 1: word-sense
- Column 2: word-form
- Column 3: index of word
- Column 4: white-space tokenized context

### Splitting the data

Your first task is to seperate the data into a *training set* and a *test set*. The training set should contain 80% of the examples and the test set the remaining 20%. The examples for the test/training set should be selected **randomly**. Save each dataset into a .csv file for loading later. **[2 marks]**

In [3]:
path = "wsd-data/wsd_data.txt"
import random



def data_split(path_to_dataset):
    
    list_of_lines = []
    with open(path_to_dataset, "r") as f:
        for line in f:
            list_of_lines.append(line)
    
    random.shuffle(list_of_lines)
    
    training = list_of_lines[:int(len(list_of_lines)*0.8)] 
    testing = list_of_lines[-int(len(list_of_lines)*0.2):]
    
    
    with open("train.csv", "w") as f:
        for line in training:
            f.write(line)
            
    with open("test.csv", "w") as f:
        for line in testing:
            f.write(line)
    
    return 


data_split(path)



### Creating a baseline

Your second task is to create a *baseline* for the task. A baseline is a "reality check" for a model, given a very simple heuristic/algorithmic/model solution to the problem, can our neural network perform better than this?
The baseline you are to create is the "most common sense" (MCS) baseline. For each word form, find the most commonly assigned sense to the word, and label a words with that sense. **[2 marks]**

E.g. In a fictional dataset, "bank" have two senses, "financial institution" which occur 5 times and "side of river" 3 times. Thus, all 8 occurences of bank is labeled "financial institution" and this yields an MCS accuracy of 5/8 = 62.5%. If a model obtain a higher score than this, we can conclude that the model *at least* is better than selecting the most frequent word sense.

In [4]:
train = "train.csv"
test = "test.csv"

from collections import Counter




def mcs_baseline(data):
    
    data = [x.strip('\n').split('\t') for x in open('train.csv')]

    listofwordsenses = []
    listofwordforms = []
    wordsense = {}

    for item in data:
        wordform = item[1]
        wordsense = item[0], item[1]
        listofwordsenses.append(wordsense)
        listofwordforms.append(wordform)
    
    
    wordsense = Counter(listofwordsenses)
    wordforms = dict.fromkeys(listofwordforms, 0)

    
    #dictionary with the highest values
    return_dict = {}


    dictcount = dict.fromkeys(listofwordforms, 0)
    for k1, v1 in wordforms.copy().items():
        for k2, v2 in wordsense.items():
            #if the word form in first dict is the same as wordform in second dict
            if k1 == k2[1]:
                #if the current value of dict1 is smaller than value of list2
                if wordforms[k1] < v2:
                    #k1 value is set to v2 value
                    wordforms[k1] = v2
                    #word label and most common word sense
                    return_dict[k1] = k2[0]
                #all values
                dictcount[k1] += v2
                
                

    for k1, v1 in wordforms.items():
        for k2, v2 in dictcount.items():
            if k1 == k2:
                wordforms[k1] = v1/v2
        
    return wordforms




baseline_dict = mcs_baseline(train)

#print(baseline_dict)
print(sum(baseline_dict.values()) / len(baseline_dict.keys()))
    
    

0.312427008668489


### Creating data iterators

To train a neural network, we first need to prepare the data. This involves converting words (and labels) to a number, and organizing the data into batches. We also want the ability to shuffle the examples such that they appear in a random order.  

To do all of this we will use the torchtext library (https://torchtext.readthedocs.io/en/latest/index.html). In addition to converting our data into numerical form and creating batches, it will generate a word and label vocabulary, and data iterators than can sort and shuffle the examples. 

Your task is to create a dataloader for the training and test set you created previously. So, how do we go about doing this?

1) First we create a ``Field`` for each of our columns. A field is a function which tokenize the input, keep a dictionary of word-to-numbers, and fix paddings. So, we need four fields, one for the word-sense, one for the position, one for the lemma and one for the context. 

2) After we have our fields, we need to process the data. For this we use the ``TabularDataset`` class. We pass the name and path of the training and test files we created previously, then we assign which field to use in each column. The result is that each column will be processed by the field indicated. So, the context column will be tokenized and processed by the context field and so on. 

3) After we have processed the dataset we need to build the vocabulary, for this we call the function ``build_vocab()`` on the different ``Fields`` with the output from ``TabularDataset`` as input. This looks at our dataset and creates the necessary vocabularies (word-to-number mappings). 

4) Finally, the last step. In the last step we load the data objects given by the ``TabularDataset`` and pass it to the ``BucketIterator`` class. This class will organize our examples into batches and shuffle them around (such that for each epoch the model observe the examples in a different order). When we are done with this we can let our function return the data iterators and vocabularies, then we are ready to train and test our model!

Implement the dataloader. [**2 marks**]

*hint: for TabularDataset and BucketIterator use the class function splits()* 

In [894]:
path = "wsd-data/"


def dataloader(path):
    whitespacer = lambda x: x.split(' ')
    
    WORDSENSE = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True) 
    
    POSITION = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True) 
    
    LEMMA = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True) 

    CONTEXT = Field(tokenize    = whitespacer,
                   lower        = True,
                   batch_first = True)

    # read the csv files
    train, test = TabularDataset.splits(path   = path,
                                        train  = 'train.csv',
                                        test   = 'test.csv',
                                        format = 'csv',
                                        fields = [('wordsense', WORDSENSE),
                                                  ("lemma", LEMMA),
                                                  ('position', POSITION),
                                                  ("context", CONTEXT)],
                                        skip_header       = True,
                                        csv_reader_params = {'delimiter':'\t',
                                                             'quotechar':'½'})

    # build vocabularies based on what our csv files contained and create word2id mapping
    WORDSENSE.build_vocab(train)
    LEMMA.build_vocab(train)
    POSITION.build_vocab(train)
    CONTEXT.build_vocab(train)

    # create batches from our data, and shuffle them for each epoch
    train_iter, test_iter = BucketIterator.splits((train, test),
                                                  batch_size        = 8,
                                                  sort_within_batch = True,
                                                  sort_key          = lambda x: len(x.context),
                                                  shuffle           = True,
                                                  device            = device)

    return train_iter, test_iter, CONTEXT, WORDSENSE, POSITION

In [900]:
train_iter, test_iter, vocab, label, POSITION = dataloader(path)

#wordsense = wordsense (duh)
#lemma = lemma of the wordsense
#position = index of the word
#context is vocabulary

#detta kan vara att du översätter nummer i data filen (e.g. 3, 4, 1) till index i din dataloader (så t.e.x. index 1 är mappat till 4 osv.). 
#Du kan fixa detta genom att antingen bygga att field som inte använder sig av ett vokabulär, 
#eller så översätter du batch.position genom nått i still med batch_positions = [IDX_VOCAB.vocab.itos[x] for x in batch.positions]


#index x mappas till en siffra i vocab
#antingen bygga ett field utan vocab
#eller översätta batch.positions från dess namn i vocab till dens egentliga namn


print(POSITION.vocab.itos[61])
print(vocab.vocab.itos[71])


71
may


# 2.1 Creating and running a Neural Network for WSD

In this section we will create and run a neural network to predict word senses based on *contextualized representations*.

### Model

We will use a bidirectional Long-Short-Term Memory (LSTM) network to create a representation for the sentences and a Linear classifier to predict the sense of each word.

When we initialize the model, we need a few things:

    1) An embedding layer: a dictionary from which we can obtain word embeddings
    2) A LSTM-module to obtain contextual representations
    3) A classifier that compute scores for each word-sense given *some* input


The general procedure is the following:

    1) For each word in the sentence, obtain word embeddings
    2) Run the embedded sentences through the RNN
    3) Select the appropriate hidden state
    4) Predict the word-sense 

**Suggestion for efficiency:**  *Use a low dimensionality (32) for word embeddings and the LSTM when developing and testing the code, then scale up when running the full training/tests*
    
Your tasks will be to create two different models (both follow the two outlines described above), described below:

In the first approach to WSD, you are to select the index of our target word (column 3 in the dataset) and predict the word sense. **[5 marks]**


In [807]:
x = torch.tensor([[[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]],
                  [[111, 124, 143],
                   [14, 15, 16],
                   [17, 18, 19]],
                 [[555, 524, 543],
                   [514, 515, 516],
                   [517, 518, 519]]])

idx = torch.tensor([0, 1, 2])            #each number = each batch, actual number = which row, all columns are used

print("index shape \t", idx.shape)
print("---------------------------------")
print("first shape \t", x.shape)
x = x[torch.arange(x.size(0)), idx]
print("---------------------------------")
print("after row selection \t", x)
print("---------------------------------")
print("shape after row selection \t", x.shape)


index shape 	 torch.Size([3])
---------------------------------
first shape 	 torch.Size([3, 3, 3])
---------------------------------
after row selection 	 tensor([[  1,   2,   3],
        [ 14,  15,  16],
        [517, 518, 519]])
---------------------------------
shape after row selection 	 torch.Size([3, 3])


för approach 1: vi behöver indexera output med batch.position för att välja representation för det flertydiga ordet, så istället för output = output[:,-1,:] så behövs nått i stil med output = output[:,position,:] (det kan vara lite klurigt att indexera en viss dimension i en tensor, men om du googlar runt lite så finns lösningen :))


detta kan vara att du översätter nummer i data filen (e.g. 3, 4, 1) till index i din dataloader (så t.e.x. index 1 är mappat till 4 osv.). Du kan fixa detta genom att antingen bygga att field som inte använder sig av ett vokabulär, eller så översätter du batch.position genom nått i still med batch_positions = [IDX_VOCAB.vocab.itos[x] for x in batch.positions]

In [911]:
class WSDModel_approach1(nn.Module):
    def __init__(self, vocab_size, wordsense_size,  emb_dim, out_dim):
        super(WSDModel_approach1, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, emb_dim)
        self.LSTM = nn.LSTM(emb_dim, out_dim, batch_first = True, bidirectional = True)          #  run embedding through rnn(LSTM is a type of rnn)
        self.classifier = nn.Linear(out_dim*2, wordsense_size)

    def forward(self, batch, word_index):                      #the batch are tensors of numbers representing words, word index = word index representing a word

        embedded_batch = self.embeddings(batch)   
        contextualized_embedding, (hn, cn) = self.LSTM(embedded_batch)   
        output = contextualized_embedding[torch.arange(contextualized_embedding.size(0)), word_index]     
        output = self.classifier(output)
        
        print("contextualized embedding shape", contextualized_embedding.shape)
        print("output shape", output.shape)
        
 
        return output


In the second approach to WSD, you are to predict the word sense based on the final hidden state given by the RNN. **[5 marks]**

In [798]:


#Att tänka på i approach 2: använd final_hidden (state[0] i din kod) och inte ord representationen för det sista ordet


class WSDModel_approach2(nn.Module):
    def __init__(self, vocab_size, wordsense_size,  emb_dim, out_dim):              
        super(WSDModel_approach2, self).__init__()                 
        self.embeddings = nn.Embedding(vocab_size, emb_dim)                 # Vocab => Embedding dimension, embedding dim is trainable  
        self.LSTM = nn.LSTM(emb_dim, out_dim,                               #
                            batch_first = True, bidirectional = True)       # run embedding through rnn(LSTM is a type of rnn)
        self.classifier = nn.Linear(out_dim*2, wordsense_size)              # 34x2 = 64, wordsense_size is 217
        
    def forward(self, batch):                         #Running the forward pass = makes a prediction
        embedded_batch = self.embeddings(batch)       # embedding layer converts word indexes to word vectors
        output, (hn, cn) = self.LSTM(embedded_batch)     # get output and state from lstm

        forward = hn[0,:,:]                            # bidirectional, l2r
        backward = hn[1,:,:]                           # bidirectional, r2l
        
        print(backward.shape, forward.shape)
        
        out = torch.cat((forward, backward), 1)        #concatenate in first dimension 
        
       # print(out.shape)
        #print(output.shape)
        #state = state[:,-1,:]                         # turns ([8, 94, 64]) into ([8, 64]), 94 is the context words
        output = self.classifier(out)              # output should be [batch size, prediction] ie [8, 217] and it is 
        

        return output

### Training and testing the model

Now we are ready to train and test our model. What we need now is a loss function, an optimizer, and our data. 

- First, create the loss function and the optimizer.
- Next, we iterate over the number of epochs (i.e. how many times we let the model see our data). 
- For each epoch, iterate over the dataset (``train_iter``) to obtain batches. Use the batch as input to the model, and let the model output scores for the different word senses.
- For each model output, calculate the loss (and print the loss) on the output and update the model parameters.
- Reset the gradients and repeat.
- After all epochs are done, test your trained model on the test set (``test_iter``) and calculate the total and per-word-form accuracy of your model.

Implement the training and testing of the model **[4 marks]**

**Suggestion for efficiency:** *when developing your model, try training and testing the model on one or two batches (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [799]:
hyperparameters = {
    'epochs':3,
    'batch_size':16,
    'learning_rate':0.001,
    'embedding_dim':32,
    'output_dim':32
}

In [912]:
train_iter, test_iter, vocab, labels, POSITION = dataloader("wsd-data/")


model1 = WSDModel_approach1(
        vocab_size = len(vocab.vocab), 
        wordsense_size = len(labels.vocab),
        emb_dim = hyperparameters["embedding_dim"],
        out_dim = hyperparameters["output_dim"])


model2 = WSDModel_approach2(
        vocab_size = len(vocab.vocab), 
        wordsense_size = len(labels.vocab),
        emb_dim = hyperparameters["embedding_dim"],
        out_dim = hyperparameters["output_dim"])



loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    model.parameters(),
    lr=hyperparameters['learning_rate']
)



total_loss = 0
for epoch in range(hyperparameters["epochs"]):
    for i, batch in enumerate(train_iter):
        

        #batch_position = torch.tensor([int(POSITION.vocab.itos[x]) for x in batch.position])
        print("batch position original", batch_position)
        batch_position = [(POSITION.vocab.itos[x]) for x in batch.position]
        print("batch_position index", batch_position)
        batch_position = [int(x) for x in batch_position]
        print("batch position as integer instead of string", batch_position)
        batch_position = torch.tensor(batch_position)
        print("batch position turned into tensor", batch_position)
        
        
        
        context = batch.context 
        word_index = batch.position
        target_word = batch.wordsense
        
        
        
        
        
        
        
       # print(batch_position)
        
        output = model1(batch.context, batch_position)
        
        
        #output = model2(context)
        
        
        #print("output shape in epochs", output.shape)
        
                
        
 
    
        loss = loss_function(output, target_word.reshape(-1))
       # loss = loss_function(output, word_index)
        total_loss += loss.item()
        
        print(total_loss/(i+1), end='\r') 
        
        
       # print("output", output.shape, "target word", target_word.reshape(-1).shape)
        
        # compute gradients
        loss.backward()
        
        # update parameters
        optimizer.step()
        
        # reset gradients
        optimizer.zero_grad()
        
    
# test model after all epochs are completed

batch position original tensor([67, 45, 58, 57, 35, 54, 62, 44])
batch_position index ['36', '67', '49', '50', '17', '56', '28', '29']
batch position as integer instead of string [36, 67, 49, 50, 17, 56, 28, 29]
batch position turned into tensor tensor([36, 67, 49, 50, 17, 56, 28, 29])
contextualized embedding shape torch.Size([8, 94, 64])
output shape torch.Size([8, 217])
batch position original tensor([36, 67, 49, 50, 17, 56, 28, 29])
batch_position index ['23', '31', '11', '13', '8', '20', '18', '11']
batch position as integer instead of string [23, 31, 11, 13, 8, 20, 18, 11]
batch position turned into tensor tensor([23, 31, 11, 13,  8, 20, 18, 11])
contextualized embedding shape torch.Size([8, 40, 64])
output shape torch.Size([8, 217])
batch position original tensor([23, 31, 11, 13,  8, 20, 18, 11])
batch_position index ['65', '60', '68', '66', '81', '37', '62', '40']
batch position as integer instead of string [65, 60, 68, 66, 81, 37, 62, 40]
batch position turned into tensor tens

contextualized embedding shape torch.Size([8, 101, 64])
output shape torch.Size([8, 217])
batch position original tensor([73, 45, 29, 38, 73, 71, 67, 36])
batch_position index ['79', '35', '93', '37', '65', '60', '15', '21']
batch position as integer instead of string [79, 35, 93, 37, 65, 60, 15, 21]
batch position turned into tensor tensor([79, 35, 93, 37, 65, 60, 15, 21])
contextualized embedding shape torch.Size([8, 130, 64])
output shape torch.Size([8, 217])
batch position original tensor([79, 35, 93, 37, 65, 60, 15, 21])
batch_position index ['33', '55', '70', '53', '73', '67', '62', '68']
batch position as integer instead of string [33, 55, 70, 53, 73, 67, 62, 68]
batch position turned into tensor tensor([33, 55, 70, 53, 73, 67, 62, 68])
contextualized embedding shape torch.Size([8, 125, 64])
output shape torch.Size([8, 217])
batch position original tensor([33, 55, 70, 53, 73, 67, 62, 68])
batch_position index ['55', '20', '63', '29', '19', '56', '62', '11']
batch position as int

batch position original tensor([20, 20, 19, 32, 24, 17, 33, 17])
batch_position index ['38', '23', '12', '21', '29', '23', '31', '8']
batch position as integer instead of string [38, 23, 12, 21, 29, 23, 31, 8]
batch position turned into tensor tensor([38, 23, 12, 21, 29, 23, 31,  8])
contextualized embedding shape torch.Size([8, 53, 64])
output shape torch.Size([8, 217])
batch position original tensor([38, 23, 12, 21, 29, 23, 31,  8])
batch_position index ['76', '41', '81', '86', '71', '33', '69', '45']
batch position as integer instead of string [76, 41, 81, 86, 71, 33, 69, 45]
batch position turned into tensor tensor([76, 41, 81, 86, 71, 33, 69, 45])
contextualized embedding shape torch.Size([8, 133, 64])
output shape torch.Size([8, 217])
batch position original tensor([76, 41, 81, 86, 71, 33, 69, 45])
batch_position index ['66', '52', '33', '51', '42', '27', '48', '23']
batch position as integer instead of string [66, 52, 33, 51, 42, 27, 48, 23]
batch position turned into tensor ten

output shape torch.Size([8, 217])
batch position original tensor([81, 68, 46, 70, 89, 55, 88, 31])
batch_position index ['78', '64', '68', '45', '73', '76', '57', '54']
batch position as integer instead of string [78, 64, 68, 45, 73, 76, 57, 54]
batch position turned into tensor tensor([78, 64, 68, 45, 73, 76, 57, 54])
contextualized embedding shape torch.Size([8, 116, 64])
output shape torch.Size([8, 217])
batch position original tensor([78, 64, 68, 45, 73, 76, 57, 54])
batch_position index ['16', '25', '41', '45', '8', '49', '39', '33']
batch position as integer instead of string [16, 25, 41, 45, 8, 49, 39, 33]
batch position turned into tensor tensor([16, 25, 41, 45,  8, 49, 39, 33])
contextualized embedding shape torch.Size([8, 76, 64])
output shape torch.Size([8, 217])
batch position original tensor([16, 25, 41, 45,  8, 49, 39, 33])
batch_position index ['19', '48', '26', '36', '52', '31', '17', '40']
batch position as integer instead of string [19, 48, 26, 36, 52, 31, 17, 40]
bat

contextualized embedding shape torch.Size([8, 87, 64])
output shape torch.Size([8, 217])
batch position original tensor([36, 15, 39, 72, 55, 37, 40, 19])
batch_position index ['34', '51', '44', '43', '19', '23', '44', '59']
batch position as integer instead of string [34, 51, 44, 43, 19, 23, 44, 59]
batch position turned into tensor tensor([34, 51, 44, 43, 19, 23, 44, 59])
contextualized embedding shape torch.Size([8, 97, 64])
output shape torch.Size([8, 217])
batch position original tensor([34, 51, 44, 43, 19, 23, 44, 59])
batch_position index ['47', '24', '18', '28', '31', '47', '38', '28']
batch position as integer instead of string [47, 24, 18, 28, 31, 47, 38, 28]
batch position turned into tensor tensor([47, 24, 18, 28, 31, 47, 38, 28])
contextualized embedding shape torch.Size([8, 67, 64])
output shape torch.Size([8, 217])
batch position original tensor([47, 24, 18, 28, 31, 47, 38, 28])
batch_position index ['54', '121', '111', '69', '61', '17', '76', '109']
batch position as int

...

batch position original tensor([50, 60, 71, 43, 45, 49, 89, 47])
batch_position index ['28', '34', '45', '11', '29', '21', '36', '33']
batch position as integer instead of string [28, 34, 45, 11, 29, 21, 36, 33]
batch position turned into tensor tensor([28, 34, 45, 11, 29, 21, 36, 33])
contextualized embedding shape torch.Size([8, 58, 64])
output shape torch.Size([8, 217])
batch position original tensor([28, 34, 45, 11, 29, 21, 36, 33])
batch_position index ['19', '38', '22', '37', '23', '11', '26', '16']
batch position as integer instead of string [19, 38, 22, 37, 23, 11, 26, 16]
batch position turned into tensor tensor([19, 38, 22, 37, 23, 11, 26, 16])
contextualized embedding shape torch.Size([8, 60, 64])
output shape torch.Size([8, 217])
batch position original tensor([19, 38, 22, 37, 23, 11, 26, 16])
batch_position index ['67', '79', '81', '102', '99', '51', '60', '60']
batch position as integer instead of string [67, 79, 81, 102, 99, 51, 60, 60]
batch position turned into tensor 

contextualized embedding shape torch.Size([8, 114, 64])
output shape torch.Size([8, 217])
batch position original tensor([49, 61, 56, 49, 33, 46, 70, 79])
batch_position index ['86', '86', '93', '18', '49', '82', '77', '57']
batch position as integer instead of string [86, 86, 93, 18, 49, 82, 77, 57]
batch position turned into tensor tensor([86, 86, 93, 18, 49, 82, 77, 57])
contextualized embedding shape torch.Size([8, 141, 64])
output shape torch.Size([8, 217])
batch position original tensor([86, 86, 93, 18, 49, 82, 77, 57])
batch_position index ['20', '33', '18', '44', '12', '48', '28', '47']
batch position as integer instead of string [20, 33, 18, 44, 12, 48, 28, 47]
batch position turned into tensor tensor([20, 33, 18, 44, 12, 48, 28, 47])
contextualized embedding shape torch.Size([8, 80, 64])
output shape torch.Size([8, 217])
batch position original tensor([20, 33, 18, 44, 12, 48, 28, 47])
batch_position index ['36', '23', '45', '33', '32', '38', '41', '36']
batch position as inte

batch position original tensor([41, 45, 77, 35, 73, 83, 21, 21])
batch_position index ['40', '65', '56', '60', '39', '26', '49', '47']
batch position as integer instead of string [40, 65, 56, 60, 39, 26, 49, 47]
batch position turned into tensor tensor([40, 65, 56, 60, 39, 26, 49, 47])
contextualized embedding shape torch.Size([8, 87, 64])
output shape torch.Size([8, 217])
batch position original tensor([40, 65, 56, 60, 39, 26, 49, 47])
batch_position index ['131', '75', '87', '96', '98', '101', '96', '76']
batch position as integer instead of string [131, 75, 87, 96, 98, 101, 96, 76]
batch position turned into tensor tensor([131,  75,  87,  96,  98, 101,  96,  76])
contextualized embedding shape torch.Size([8, 177, 64])
output shape torch.Size([8, 217])
batch position original tensor([131,  75,  87,  96,  98, 101,  96,  76])
batch_position index ['34', '24', '28', '40', '34', '28', '27', '25']
batch position as integer instead of string [34, 24, 28, 40, 34, 28, 27, 25]
batch position 

contextualized embedding shape torch.Size([8, 152, 64])
output shape torch.Size([8, 217])
batch position original tensor([ 80, 124,  99,  61,  97,  59,  86,  45])
batch_position index ['92', '127', '63', '106', '88', '75', '113', '91']
batch position as integer instead of string [92, 127, 63, 106, 88, 75, 113, 91]
batch position turned into tensor tensor([ 92, 127,  63, 106,  88,  75, 113,  91])
contextualized embedding shape torch.Size([8, 251, 64])
output shape torch.Size([8, 217])
batch position original tensor([ 92, 127,  63, 106,  88,  75, 113,  91])
batch_position index ['75', '73', '68', '53', '101', '52', '85', '66']
batch position as integer instead of string [75, 73, 68, 53, 101, 52, 85, 66]
batch position turned into tensor tensor([ 75,  73,  68,  53, 101,  52,  85,  66])
contextualized embedding shape torch.Size([8, 134, 64])
output shape torch.Size([8, 217])
batch position original tensor([ 75,  73,  68,  53, 101,  52,  85,  66])
batch_position index ['56', '42', '73', '11

IndexError: index 61 is out of bounds for dimension 1 with size 39

In [585]:
torch.save(model2, "model2")

In [324]:
torch.load("model2")

WSDModel_approach2(
  (embeddings): Embedding(17502, 32)
  (LSTM): LSTM(32, 32, batch_first=True, bidirectional=True)
  (classifier): Linear(in_features=64, out_features=217, bias=True)
)

In [336]:
## Test the model on the test_iter
test_loss = 0
# tell the model to go into evaluation mode
num_labels = len(labels.vocab)
model2.eval()
# iterate over the test data and compute the class probabilities, same
# procedure as before, but now we don't backpropagate
for i, batch in enumerate(test_iter):
    wordsense = batch.wordsense


    with torch.no_grad(): # dont collect gradients when testing
        output = model2(batch.context)
    batch_loss = loss_function(output, wordsense.reshape(-1))
    test_loss += batch_loss.item()
    

    print('>', np.round(test_loss/(i+1), 4))

tensor(5.4178)
> 5.4178
tensor(5.3775)
> 5.3976
tensor(5.3882)
> 5.3945
tensor(5.4014)
> 5.3962
tensor(5.4012)
> 5.3972
tensor(5.3744)
> 5.3934
tensor(5.4462)
> 5.401
tensor(5.3536)
> 5.395
tensor(5.3890)
> 5.3944
tensor(5.3769)
> 5.3926
tensor(5.3117)
> 5.3853
tensor(5.3591)
> 5.3831
tensor(5.4038)
> 5.3847
tensor(5.3890)
> 5.385
tensor(5.4197)
> 5.3873
tensor(5.3647)
> 5.3859
tensor(5.3931)
> 5.3863
tensor(5.3380)
> 5.3836
tensor(5.4064)
> 5.3848
tensor(5.4390)
> 5.3875
tensor(5.3918)
> 5.3877
tensor(5.3503)
> 5.386
tensor(5.4395)
> 5.3884
tensor(5.3063)
> 5.3849
tensor(5.3549)
> 5.3837
tensor(5.3450)
> 5.3823
tensor(5.3581)
> 5.3814
tensor(5.4599)
> 5.3842
tensor(5.4019)
> 5.3848
tensor(5.3254)
> 5.3828
tensor(5.4005)
> 5.3834
tensor(5.3494)
> 5.3823
tensor(5.3984)
> 5.3828
tensor(5.4088)
> 5.3836
tensor(5.3798)
> 5.3835
tensor(5.3965)
> 5.3838
tensor(5.3826)
> 5.3838
tensor(5.4030)
> 5.3843
tensor(5.4048)
> 5.3848
tensor(5.4491)
> 5.3864
tensor(5.3958)
> 5.3867
tensor(5.3987)
> 5.3

In [376]:
print(len(vocab.vocab))
print(len(labels.vocab))
#print(len(labels.vocab))

print(vocab.vocab.stoi)

nonsense_word_idx = vocab.vocab.stoi["group"]
print(f"word: {nonsense_word_idx}")
model.embeddings(torch.LongTensor([nonsense_word_idx]))

17502
217
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fbf4e43f850>>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'of': 3, ',': 4, 'and': 5, 'to': 6, '.': 7, 'in': 8, 'a': 9, 'that': 10, 'for': 11, 'on': 12, 'with': 13, 'as': 14, '(': 15, 'is': 16, 'by': 17, ')': 18, 'be': 19, 'it': 20, 'or': 21, 'its': 22, 'was': 23, 'at': 24, 'are': 25, 'united': 26, 'which': 27, ';': 28, 'this': 29, 'an': 30, 'not': 31, "'s": 32, 'international': 33, 'from': 34, 'nations': 35, 'have': 36, 'their': 37, 'has': 38, 'all': 39, 'also': 40, 'been': 41, 'other': 42, 'had': 43, 'security': 44, 'we': 45, 'rights': 46, 'states': 47, 'would': 48, 'development': 49, 'countries': 50, 'were': 51, 'such': 52, 'will': 53, 'women': 54, 'should': 55, 'committee': 56, 'national': 57, '“': 58, 'council': 59, '”': 60, 'human': 61, ':': 62, 'government': 63, 'law': 64, 'state': 65, 'under': 66, 'his': 67, 'those': 68, 'order': 69, 'including': 70, 'may': 71, 'they': 72, 'work': 73, 'mo

tensor([[ 0.2568, -2.8253, -0.0587, -1.4233,  1.2224, -0.2735, -0.6024, -0.7125,
          0.0411,  0.7548, -1.0888, -0.7712,  1.1456, -0.4631, -0.4470, -1.7161,
          1.3275, -1.7873, -0.6655, -0.3130, -0.0740,  0.7142, -2.1404,  1.8623,
          0.4715,  0.3914, -1.3531, -0.2882, -1.3030,  0.0446,  0.9904,  1.7833]],
       grad_fn=<EmbeddingBackward>)

# 2.2 Running a transformer for WSD

In this section of the lab you'll try out the transformer, specifically the BERT model. For this we'll use the huggingface library (https://huggingface.co/).

You can find the documentation for the BERT model here (https://huggingface.co/transformers/model_doc/bert.html) and a general usage guide here (https://huggingface.co/transformers/quickstart.html).

What we're going to do is *fine-tune* the BERT model, i.e. update the weights of a pre-trained model. That is, we have a model that is trained on language modeling, but now we apply it to word sense disambiguation with the word representations it learnt from language modeling.

We'll use the same data splits for training and testing as before, but this time you'll not use a torchtext dataloader. Rather now you create an iterator that collects N sentences (where N is the batch size) then use the BertTokenizer to transform the sentence into integers. For your dataloader, remember to:
* Shuffle the data in each batch
* Make sure you get a new iterator for each *epoch*
* Create a vocabulary of *sense-labels* so you can calculate accuracy 

We then pass this batch into the BERT model and train as before. The BERT model will encode the sentence, then we send this encoded sentence into a prediction layer (you can either the the sentence-representation from bert, or the ambiguous word) like before and collect sense predictions.

About the hyperparameters and training:
* For BERT, usually a lower learning rate works best, between 0.0001-0.000001.
* BERT takes alot of resources, running it on CPU will take ages, utilize the GPUs :)
* Since BERT takes alot of resources, use a small batch size (4-8)
* Computing the BERT representation, make sure you pass the mask

**[10 marks]**

In [ ]:
def dataloader_for_bert(path_to_file, batch_size):
    ...

In [ ]:
class BERT_WSD(nn.Module):
    def __init__(self, ...):
        # your code goes here
        self.bert = ...
        self.classifier = ...
    
    def forward(self, batch):
        # your code goes here
        
        return predictions

In [ ]:
loss_function = ...
optimizer = ...
model = ...

for _ in range(epochs):
    # train model
    ...
    
# test model after all epochs are completed

# 3. Evaluation

Explain the difference between the first and second approach. What kind of representations are the different approaches using to predict word-senses? **[4 marks]**

Evaluate your model with per-word-form *accuracy* and comment on the results you get, how does the model perform in comparison to the baseline, and how do the models compare to each other? 

Expand on the evaluation by sorting the word-forms by the number of senses they have. Are word-forms with fewer senses easier to predict? Give a short explanation of the results you get based on the number of senses per word.

**[6 marks]**

How does the LSTMs perform in comparison to BERT? What's the difference between representations obtained by the LSTMs and BERT? **[2 marks]**

What could we do to improve our LSTM word sense disambiguation models and our BERT model? **[4 marks]**

# Readings:

[1] Kågebäck, M., & Salomonsson, H. (2016). Word Sense Disambiguation using a Bidirectional LSTM. arXiv preprint arXiv:1606.03568.

[2] https://cl.lingfil.uu.se/~nivre/master/NLP-LexSem.pdf